In [1]:
import warnings
warnings.filterwarnings("ignore")

from data_loader import load_data_from_disk, process_to_df, cut_light_change
from draw import plot_per_light_cuts, plot_pupil_ci, plot_pupil_ci_subplots, rgb_color_names

import os
import matplotlib.pyplot as plt

In [2]:
TEST_GROUP={
    "type" : "单色1",
    "time" : 1,
    "start" : 1,
    "duration" : 3,
    "step" : 2
}
# TEST_GROUP={
#     "type" : "单色3",
#     "time" : 3,
#     "start" : 3,
#     "duration" : 9,
#     "step" : 6
# }
# TEST_GROUP={
#     "type" : "单色5",
#     "time" : 5,
#     "start" : 5,
#     "duration" : 15,
#     "step" : 10
# }
# TEST_GROUP={
#     "type" : "单色p5",
#     "time" : 0.5,
#     "start" : 0.5,
#     "duration" : 1.5,
#     "step" : 1
# }

In [3]:
if not os.path.exists(f"exp-1-pic\\{TEST_GROUP['type']}"):
    os.mkdir(f"exp-1-pic\\{TEST_GROUP['type']}")

light_data = load_data_from_disk(
    "E:\LightYourEyesData\实验一",
    data_type="light_changes",
    test_type=TEST_GROUP['type'],
    test_times=1
)

light_df = process_to_df(
    light_data,
    smooth_win=int(0.5*TEST_GROUP['time']*50)
)

In [4]:
cutted_data = cut_light_change(
    light_df,
    start_time=TEST_GROUP['start'],
    duration=TEST_GROUP['duration'],
    step=TEST_GROUP['step']
)

In [11]:
cutted_data[0]

{'start_rgb': [255.0, 255.0, 255.0],
 'start_energy': 224.85799206684482,
 'end_rgb': [255.0, 128.0, 0.0],
 'end_energy': 112.945706598837,
 'data':         leftPD    rightPD videoName         skyboxColorRGB
 50  -38.093708 -31.196836            [255.0, 255.0, 255.0]
 51  -38.223228 -30.045446            [255.0, 255.0, 255.0]
 52  -38.303042 -28.937371            [255.0, 255.0, 255.0]
 53  -38.356651 -27.988997            [255.0, 255.0, 255.0]
 54  -38.412879 -27.306945            [255.0, 255.0, 255.0]
 ..         ...        ...       ...                    ...
 195 -46.572099 -44.897775              [255.0, 128.0, 0.0]
 196 -46.456761 -44.884134              [255.0, 128.0, 0.0]
 197 -46.330288 -44.852681              [255.0, 128.0, 0.0]
 198 -46.196781 -44.801959              [255.0, 128.0, 0.0]
 199 -46.058335 -44.732369              [255.0, 128.0, 0.0]
 
 [150 rows x 4 columns]}

In [ ]:
from metrics import analyze_pupil_response
from Light import rgb_to_energy_spectral, str_to_rgb

res = {}
for idx, cut in enumerate(cutted_data):
    temp_df = cut['data']
    temp = (temp_df["leftPD"].values + temp_df["rightPD"].values) / 2
    ana_res = analyze_pupil_response(
        pupil_data=temp,
        sample_rate=50,
        stimulus_onset_frame=int((TEST_GROUP['time']+0.2)*50),
        color_terminal_frame=int((TEST_GROUP['time']*2)*50),
        std_threshold=1
    )

    energy_trans_vel = (cut["end_energy"] - cut["start_energy"]) / 

    res[idx] = ana_res


In [10]:
import pandas as pd

pd.DataFrame(res).T.describe()

,latency_ms,latency_onset_time,average_transition_speed
count,289.000000,289.000000,234.000000
mean,529.342561,0.529343,20.491626
std,364.767692,0.364768,12.334725
min,0.000000,0.000000,3.898079
25%,280.000000,0.280000,12.107398
50%,480.000000,0.480000,17.549406
75%,720.000000,0.720000,25.029284
max,1660.000000,1.660000,72.048738


In [6]:
samestart_cluster = {}

for cut in cutted_data:
    s_rgb = "-".join(str(n) for n in cut["start_rgb"])
    e_rgb = "-".join(str(n) for n in cut["end_rgb"])
    if s_rgb in samestart_cluster:
        if e_rgb in samestart_cluster[s_rgb]:
            samestart_cluster[s_rgb][e_rgb].append(cut["data"])
        else:
            samestart_cluster[s_rgb][e_rgb] = [cut["data"]]
    else:
        samestart_cluster[s_rgb] = {}
        samestart_cluster[s_rgb][e_rgb] = [cut["data"]]

In [ ]:
import numpy as np

trans_subtraction = {}
for start in samestart_cluster.keys():
    if start not in rgb_color_names.keys(): continue
    trans_subtraction[rgb_color_names[start]] = {}
    for end in samestart_cluster[start].keys():
        subs = []
        for _d in samestart_cluster[start][end]:
            trans_s = int(TEST_GROUP["time"]*50)
            trans_e = int(TEST_GROUP["time"]*2*50)

            subs.append(np.mean(_d["leftPD"][trans_e-3 :trans_e+4]) - np.mean(_d["leftPD"][trans_s-3 : trans_s+4]))
            subs.append(np.mean(_d["rightPD"][trans_e-3 :trans_e+4]) - np.mean(_d["rightPD"][trans_s-3 : trans_s+4]))
        # trans_subtraction[rgb_color_names[start]][rgb_color_names[end]] = np.mean(subs)
        trans_subtraction[rgb_color_names[start]][rgb_color_names[end]] = subs

In [8]:
import pandas as pd
from draw import rgb_color_names
trans_diff = pd.DataFrame(trans_subtraction)
# trans_diff.index = [rgb_color_names[i] for i in trans_diff.index]
# trans_diff.columns = [rgb_color_names[i] for i in trans_diff.columns]
trans_diff.to_csv(os.path.join(f"exp-1-pic\\{TEST_GROUP['type']}-transdiff.csv"))

In [9]:
from draw import rgb_color_names
from Light import rgb_to_energy_spectral, str_to_rgb
rgb_energydiff = {}

for s_rgb in rgb_color_names:
    rgb_energydiff[rgb_color_names[s_rgb]] = {}
    for e_rgb in rgb_color_names:
        _sub = rgb_to_energy_spectral(str_to_rgb(e_rgb)) - rgb_to_energy_spectral(str_to_rgb(s_rgb))
        rgb_energydiff[rgb_color_names[s_rgb]][rgb_color_names[e_rgb]] = _sub

In [13]:
from Light import rgb_to_energy_spectral, str_to_rgb

energydiff_cluster = {
    "< -100":[],
    "-100 ~ -50":[],
    "-50 ~ -25":[],
    "-25 ~ 25":[],
    "25 ~ 50":[],
    "50 ~ 100":[],
    "> 100":[]
}

for cut in cutted_data:
    s_rgb = "-".join(str(n) for n in cut["start_rgb"])
    e_rgb = "-".join(str(n) for n in cut["end_rgb"])
        
    _sub = rgb_to_energy_spectral(str_to_rgb(e_rgb)) - rgb_to_energy_spectral(str_to_rgb(s_rgb))
    if _sub<-100:
        energydiff_cluster["< -100"].append(cut["data"])
    elif -100 <= _sub < -50:
        energydiff_cluster["-100 ~ -50"].append(cut["data"])
    elif -50 <= _sub < -25:
        energydiff_cluster["-50 ~ -25"].append(cut["data"])
    elif -25 <= _sub < 25:
        energydiff_cluster["-25 ~ 25"].append(cut["data"])
    elif 25 <= _sub < 50:
        energydiff_cluster["25 ~ 50"].append(cut["data"])
    elif 50 <= _sub < 100:
        energydiff_cluster["50 ~ 100"].append(cut["data"])
    elif 100 <= _sub:
        energydiff_cluster["> 100"].append(cut["data"])

In [16]:
from Light import rgb_to_str

def cut_by_stage(data):
    df = data.copy()
    df["skyboxColorRGB"] = df["skyboxColorRGB"].apply(rgb_to_str)
    
    # Initialize the dictionary to hold the results
    phases = {"stay": [], "transition": []}
    
    # Convert skyboxcolor to a categorical type for efficient comparison
    color_series = df['skyboxColorRGB']
    
    # Find the indices where the color changes
    transition_indices = color_series[color_series.shift(2) != color_series].index
    
    s_idx = transition_indices[0]
    e_idx = transition_indices[1]
    _idx = 1
    while _idx<len(transition_indices):
        if transition_indices[_idx] - transition_indices[_idx-1]==1:
            e_idx = transition_indices[_idx]
        else:
            phases["transition"].append(df.iloc[s_idx:e_idx+1].copy())
            phases["stay"].append(df.iloc[e_idx+1:transition_indices[_idx]].copy())
            s_idx = transition_indices[_idx]
            e_idx = transition_indices[_idx]
        _idx += 1
    phases["transition"].append(df.iloc[s_idx:transition_indices[-1]+1].copy())
    phases["stay"].append(df.iloc[transition_indices[-1]+1:].copy())
            
    return phases